### [An Explanation of the Deflate Algorithm](https://zlib.net/feldspar.html)

* [RFC 1951](https://www.ietf.org/rfc/rfc1951.txt)
* [Huffman Coding - Python Implementation](http://bhrigu.me/blog/2017/01/17/huffman-coding-python-implementation/)

In [1]:
%run ~/Oxebfe/FileProcessor/Module/Utilities.ipynb
%run ~/Oxebfe/FileProcessor/Module/LZ77.ipynb
%run ~/Oxebfe/FileProcessor/Module/HuffmanCoding.ipynb

#### Inflate Decompress
* Compression Blocks
* Deflate Decompress No Compression
* Deflate Decompress Fixed Huffman Code
* Deflate Decompress Dynamic Huffman Code

#### Deflate Compress
* Compressiono Blocks
* Deflate Decompress No Compression
* Deflate Decompress Fixed Huffman Code
* Deflate Decompress Dynamic Huffman Code

In [12]:
def ParseCodeLengthOfCodeLengthSymbols(bits):
    assert type(bits) == list and len(bits) % 3 == 0
    CodeLengthSymbolOrder = [16, 17, 18, 0, 8, 7, 9, 6, 10, 5, 11, 4, 12, 3, 13, 2, 14, 1, 15]
    result = list()
    n = len(bits) // 3
    assert n <= len(CodeLengthSymbolOrder)
    for i in range(0, n):
        symbol = "{:02d}".format(CodeLengthSymbolOrder[i])
        code_length = BitsToInt(bits[3*i:3*(i+1)], "lsb")
        if code_length > 0:
            result.append([symbol, code_length])
    result.sort(key=lambda x: x[0])
    return result
    
def GenCodeLengthHuffmanDecodingTable(code_length_of_code_length_symbols):
    table = list()
    for (symbol, code_length) in code_length_of_code_length_symbols:
        new = True
        for item in table:
            if item["code_length"] == code_length:
                item["symbols"].append(symbol)
                new = False
                break
        if new:
            temp = {"code_length": code_length, "symbols": [symbol]}
            table.append(temp)
    table.sort(key=lambda x: x["code_length"])
    n = len(table)
    for i in range(0, n):
        item = table[i]
        if i == 0:
            item["init_value"] = 0
        else:
            prev_item = table[i-1]
            item["init_value"] = (prev_item["init_value"] + len(prev_item["symbols"])) << (item["code_length"] - prev_item["code_length"])
    result = dict()
    for i in range(0, n):
        item = table[i]
        m = len(item["symbols"])
        for j in range(0, m):
            symbol = item["symbols"][j]
            form = "{{:0{}b}}".format(item["code_length"])
            value = form.format(item["init_value"]+j)
            result[value] = int(symbol, 10)    
    return result

def GenHuffmanDecodingTable(code_length_of_symbols):
    table = list()
    n = len(code_length_of_symbols)
    for i in range(0, n):
        if code_length_of_symbols[i] != 0:
            new = True
            for item in table:
                if item["code_length"] == code_length_of_symbols[i]:
                    item["symbols"].append(i)
                    new = False
                    break
            if new:
                temp = {"code_length": code_length_of_symbols[i], "symbols": [i]}
                table.append(temp)
    table.sort(key=lambda x: x["code_length"])
    n = len(table)
    for i in range(0, n):
        item = table[i]
        if i == 0:
            item["init_value"] = 0
        else:
            prev_item = table[i-1]
            item["init_value"] = (prev_item["init_value"] + len(prev_item["symbols"])) << (item["code_length"] - prev_item["code_length"])
    result = dict()
    for i in range(0, n):
        item = table[i]
        m = len(item["symbols"])
        for j in range(0, m):
            symbol = item["symbols"][j]
            form = "{{:0{}b}}".format(item["code_length"])
            value = form.format(item["init_value"]+j)
            result[value] = symbol
    return result

def GenFixedHuffmanDecodingTable():
    table = [
        {"code_length": 7, "symbols": list(range(256, 280)), "init_value": 0},
        {"code_length": 8, "symbols": list(range(0, 144)) + list(range(280, 288)), "init_value": 48},
        {"code_length": 9, "symbols": list(range(144, 256)), "init_value": 400}
    ]
    n = len(table)
    for i in range(0, n):
        item = table[i]
        if i == 0:
            item["init_value"] = 0
        else:
            prev_item = table[i-1]
            item["init_value"] = (prev_item["init_value"] + len(prev_item["symbols"])) << (item["code_length"] - prev_item["code_length"])
    result = dict()
    for i in range(0, n):
        item = table[i]
        m = len(item["symbols"])
        for j in range(0, m):
            symbol = item["symbols"][j]
            form = "{{:0{}b}}".format(item["code_length"])
            value = form.format(item["init_value"]+j)
            result[value] = symbol
    return result    

def inflate(data):
    assert type(data) == bytes
    bits = BytesToBits(data, "lsb")
    decompressed = bytes()
    index = 0
    last = False
    while not last:
        last = bits[index]
        index += 1
        method = BitsToInt(bits[index:index+2], "lsb")
        index += 2
        assert method in range(0, 3)
        if method == 0:# no compression
            index += 8 - (index % 8) if index % 8 != 0 else 0
            length_0 = BitsToInt(bits[index:index+16], "lsb")
            index += 16
            length_1 = BitsToInt(bits[index:index+16], "lsb")
            index += 16
            assert length_0 + length_1 == (1<<16)
            for i in range(0, length_0):
                temp = BitsToInt(bits[index:index+8], "lsb")
                index += 8
                byte = temp.to_btyes(1, "little")
                decompressed += byte
        elif method == 1:# fixed Huffman code
            result_2 = GenFixedHuffmanDecodingTable()
            result_4 = list()
            while index < len(bits):
                temp = str(bits[index])
                index += 1
                while temp not in result_2.keys():
                    temp += str(bits[index])
                    index += 1
                symbol = result_2[temp]
                assert symbol in range(0, 286)
                if symbol < 256:
                    result_4.append(symbol.to_bytes(1, "little"))
                elif symbol == 256:
                    result_4.append("EOB")
                    break
                else:
                    if symbol in range(257, 265):
                        length = symbol - 254
                    elif symbol in range(265, 285):
                        extra = (symbol - 261) // 4
                        length = 3 + (1<<(extra+2)) + ((symbol - 261) % 4)*(1<<extra) + BitsToInt(bits[index:index+extra], "lsb")
                        index += extra
                    elif symbol == 285:
                        length = 258
                    else:
                        break
                    symbol = BitsToInt(bits[index:index+5], "msb")
                    index += 5
                    assert symbol in range(0, 30)
                    if symbol in range(0, 4):
                        distance = symbol + 1
                    elif symbol in range(4, 30):
                        extra = (symbol - 2) // 2
                        distance = 1 + (1<<(extra+1)) + ((symbol - 2) % 2)*(1<<extra) + BitsToInt(bits[index:index+extra], "lsb")
                        index += extra
                    else:
                        break
                    result_4.append((length, distance))
            decompressed += LZ77Decode(result_4, decompressed)
        else:# method == 2, dynamic Huffman code
            hlit = BitsToInt(bits[index:index+5], "lsb") + 257
            index += 5
            hdist = BitsToInt(bits[index:index+5], "lsb") + 1
            index += 5
            hclen = BitsToInt(bits[index:index+4], "lsb") + 4
            index += 4
            result_0 = ParseCodeLengthOfCodeLengthSymbols(bits[index:index+3*hclen])
            index += 3*hclen
            table = GenCodeLengthHuffmanDecodingTable(result_0)            
            result_1 = list()
            while len(result_1) < hlit + hdist:
                temp = str(bits[index])
                index += 1
                while temp not in table.keys():
                    temp += str(bits[index])
                    index += 1
                value = table[temp]
                if value == 16:
                    count = 3 + BitsToInt(bits[index:index+2], "lsb")
                    index += 2
                    value = result_1[len(result_1)-1]
                    for i in range(0, count):
                        result_1.append(value)
                elif value == 17:
                    count = 3 + BitsToInt(bits[index:index+3], "lsb")
                    index += 3
                    value = 0
                    for i in range(0, count):
                        result_1.append(value)
                elif value == 18:
                    count = 11 + BitsToInt(bits[index:index+7], "lsb")
                    index += 7
                    value = 0
                    for i in range(0, count):
                        result_1.append(value)
                else:
                    result_1.append(value)
            assert len(result_1) == hlit + hdist
            result_2 = GenHuffmanDecodingTable(result_1[0:hlit])
            result_3 = GenHuffmanDecodingTable(result_1[hlit:hlit+hdist])
            result_4 = list()
            while index < len(bits):
                temp = str(bits[index])
                index += 1
                while temp not in result_2.keys():
                    temp += str(bits[index])
                    index += 1
                symbol = result_2[temp]
                assert symbol in range(0, 286)
                if symbol < 256:
                    result_4.append(symbol.to_bytes(1, "little"))
                elif symbol == 256:
                    result_4.append("EOB")
                    break
                else:
                    if symbol in range(257, 265):
                        length = symbol - 254
                    elif symbol in range(265, 285):
                        extra = (symbol - 261) // 4
                        length = 3 + (1<<(extra+2)) + ((symbol - 261) % 4)*(1<<extra) + BitsToInt(bits[index:index+extra], "lsb")
                        index += extra
                    elif symbol == 285:
                        length = 258
                    else:
                        break
                    temp = str(bits[index])
                    index += 1
                    while temp not in result_3.keys():
                        temp += str(bits[index])
                        index += 1
                    symbol = result_3[temp]
                    assert symbol in range(0, 30)
                    if symbol in range(0, 4):
                        distance = symbol + 1
                    elif symbol in range(4, 30):
                        extra = (symbol - 2) // 2
                        distance = 1 + (1<<(extra+1)) + ((symbol - 2) % 2)*(1<<extra) + BitsToInt(bits[index:index+extra], "lsb")
                        index += extra
                    else:
                        break
                    result_4.append((length, distance))
            decompressed += LZ77Decode(result_4, decompressed)
    return decompressed         

In [16]:
def FixedHuffmanEncode(symbol):
    assert type(symbol) in [bytes, tuple]
    bits = list()
    if type(symbol) == bytes:
        assert len(symbol) == 1
        value = int.from_bytes(symbol, "little")
        if value in range(0, 144):
            temp = value + 48
            for i in range(0, 8):
                bit = (temp & 1<<(7-i))>>(7-i)
                bits.append(bit)
        if value in range(144, 256):
            temp = value - 144 + 16
            bits = [1, 1]
            for i in range(0, 7):
                bit = (temp & 1<<(6-i))>>(6-i)
                bits.append(bit)
    else:#type(symbol) == tuple
        assert len(symbol) == 2
        (length, distance) = symbol
        assert length in range(3, 259)
        if length in range(3, 11):
            value = length + 254
            extra = list()
        elif length in range(11, 258):
            value = length + 254
            extra = list()            
        else:
            value = 285
            extra = list()
        if value in range(257, 280):
            temp = value - 256
            bits = [0, 0]
            for i in range(0, 5):
                bit = (temp & 1<<(4-i))>>(4-i)
                bits.append(bit)
            bits.append(extra)
        elif value in range(280, 288):
            temp = value - 280
            bits = [1, 1, 0, 0, 0]
            for i in range(0, 3):
                bit = (temp & 1<<(2-i))>>(2-i)
                bits.append(bit)
            bits.append(extra)
        assert distance in range(1, 32769)
        
    return bits

def deflate(data, mode=1):
    assert type(data) == bytes and mode in [1, 2]
    codes = LZ77Encode(data)
    result = [1] #last block
    if mode == 1:
        result += [1, 0]
        for symbol in codes:
            temp = FixedHuffmanEncode(symbol)
            result += temp
        n = 14 - ((len(result) - 2) % 8)
        for i in range(0, n):
            result.append(0)
        assert len(result) % 8 == 0
    else:
        print("Not Implemented!")
    return result